In [1]:
"""
WITH highway_weather_station_dist AS (
    SELECT
        h.line_num AS highway_id,
        w.id AS weather_station_id,
        ST_Distance(h.geom, w.geom) AS distance,
        ROW_NUMBER() OVER (PARTITION BY h.line_num ORDER BY ST_Distance(h.geom, w.geom)) as rn
    FROM
        (SELECT line_num, ST_SetSRID(ST_MakePoint(longitude, latitude), 4326) AS geom FROM highway_coord) h,
        (SELECT id, ST_SetSRID(ST_MakePoint(longitude, latitude), 4326) AS geom FROM weather_station_coord WHERE id in(108, 101, 104, 232, 127, 130, 146, 135, 138, 156, 285, 159, 295, 184)) w
)

SELECT *
FROM 
    (SELECT highway_id, weather_station_id FROM highway_weather_station_dist WHERE rn = 1) nearest_ws JOIN
    (SELECT tm, id, location, rain, snow, cloud, weather_code FROM hourly_weather_data) hwd
    ON nearest_ws.weather_station_id = hwd.id
ORDER BY nearest_ws.highway_id;

"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

hw_loc = pd.read_csv("./highway_ic_coord.csv", sep=';').drop(columns = ['color'])
weather = pd.read_csv("./weather_data_20240101_20240514_full.csv")
ws_loc = pd.read_csv("./weather_station_long_lat.csv")
hw_loc

,name,path_json
0,경부선(경부선시점-구서IC),"[[129.09320677, 35.24657831], [129.093941182, ..."
1,경부선(구서IC-영락IC),"[[129.094675607, 35.247919117], [129.095410044..."
2,경부선(영락IC-부산TG),"[[129.104050468, 35.261976637], [129.104601332..."
3,경부선(부산TG-노포IC),"[[129.106609683, 35.278962993], [129.10614843,..."
4,경부선(노포IC-노포JC),"[[129.100226401, 35.287170823], [129.099330599..."
...,...,...
496,대구외곽순환선(파군재IC-둔산IC),"[[128.64389127, 35.935676544], [128.644864426,..."
497,대구외곽순환선(둔산IC-상매JCT),"[[128.694072021, 35.900104288], [128.695092451..."
498,광주외곽순환선(남광산IC-남광산TG),"[[126.747842548, 35.184788845], [126.748594392..."
499,광주외곽순환선(남광산TG-남장성IC),"[[126.755637043, 35.191135394], [126.756423897..."


In [2]:
hw_loc.loc[:, "longitude"] = hw_loc.loc[:,"path_json"].map(lambda x: np.vstack(eval(x), dtype=np.float64).mean(axis = 0)[0])
hw_loc.loc[:, "latitude"] = hw_loc.loc[:,"path_json"].map(lambda x: np.vstack(eval(x), dtype=np.float64).mean(axis = 0)[1])
hw_loc = hw_loc.drop(columns = ["path_json"])
hw_loc.columns = ['section', 'longitude', 'latitude']
hw_loc.to_csv("./highway_ic_mean_coord.csv", index = False)
hw_loc

,section,longitude,latitude
0,경부선(경부선시점-구서IC),129.093574,35.246914
1,경부선(구서IC-영락IC),129.099711,35.254265
2,경부선(영락IC-부산TG),129.107086,35.269997
3,경부선(부산TG-노포IC),129.104157,35.282976
4,경부선(노포IC-노포JC),129.098435,35.288215
...,...,...,...
496,대구외곽순환선(파군재IC-둔산IC),128.669489,35.918531
497,대구외곽순환선(둔산IC-상매JCT),128.699511,35.895640
498,광주외곽순환선(남광산IC-남광산TG),126.751322,35.187667
499,광주외곽순환선(남광산TG-남장성IC),126.771977,35.208510


In [4]:
import pandas as pd
from geopy.distance import geodesic


# 거리 계산 함수
def find_nearest(lat, lon, df):
    distances = df.apply(lambda row: geodesic((lat, lon), (row['latitude'], row['longitude'])).kilometers, axis=1)
    return df.loc[distances.idxmin()]

# 각 점에 대해 가장 가까운 점 찾기
nearest_points = hw_loc.apply(lambda row: find_nearest(row['latitude'], row['longitude'], ws_loc), axis=1)

# 결과를 원본 df1에 합병
result = pd.concat([hw_loc, nearest_points], axis=1)
highway_nearest_ws = result.drop(columns=['latitude', 'longitude'])
highway_nearest_ws.to_csv("./highway_nearest_ws.csv", index = False)
highway_nearest_ws.head()

,section,weather_station_id,weather_station_name
0,경부선(경부선시점-구서IC),257,양산시
1,경부선(구서IC-영락IC),257,양산시
2,경부선(영락IC-부산TG),257,양산시
3,경부선(부산TG-노포IC),257,양산시
4,경부선(노포IC-노포JC),257,양산시


In [5]:
hourly_highway_weather = pd.merge(highway_nearest_ws, weather, left_on='weather_station_id', right_on='id').drop(columns = ['location', 'id'])
# hourly_highway_weather.columns = ['section', 'weather_station_id', 'weather_station_name', 'tm', 'rain', 'snow', 'cloud', 'weather_code']
hourly_highway_weather.to_csv("./hourly_highway_weather.csv", index = False)
hourly_highway_weather

,section,weather_station_id,weather_station_name,tm,rain,snow,cloud,weather_code
0,경부선(경부선시점-구서IC),257,양산시,2024-01-01 00:00:00,0.0,0.0,0,NaN
1,경부선(경부선시점-구서IC),257,양산시,2024-01-01 01:00:00,0.0,0.0,0,NaN
2,경부선(경부선시점-구서IC),257,양산시,2024-01-01 02:00:00,0.0,0.0,0,NaN
3,경부선(경부선시점-구서IC),257,양산시,2024-01-01 03:00:00,0.0,0.0,5,NaN
4,경부선(경부선시점-구서IC),257,양산시,2024-01-01 04:00:00,0.0,0.0,6,NaN
...,...,...,...,...,...,...,...,...
1611211,광주외곽순환선(남장성IC-남장성JCT),156,광주,2024-05-13 19:00:00,0.0,0.0,0,NaN
1611212,광주외곽순환선(남장성IC-남장성JCT),156,광주,2024-05-13 20:00:00,0.0,0.0,0,NaN
1611213,광주외곽순환선(남장성IC-남장성JCT),156,광주,2024-05-13 21:00:00,0.0,0.0,0,NaN
1611214,광주외곽순환선(남장성IC-남장성JCT),156,광주,2024-05-13 22:00:00,0.0,0.0,0,NaN


In [6]:
hourly_highway_weather.loc[hourly_highway_weather['section'].map(lambda x: x.startswith('경부선')), :]['weather_station_name'].unique()

array(['양산시', '울산', '경주시', '영천', '대구', '구미', '추풍령', '대전', '청주', '서청주',
       '천안', '수원', '서울'], dtype=object)